Anticipez les besoins en consommation électrique de bâtiments
=============================================================

![logo-seattle](../reports/figures/logo-seattle.png)


Explication des variables:
[City of seattle](https://data.seattle.gov/dataset/2015-Building-Energy-Benchmarking/h7rm-fz6m)

## Importation des librairies

In [1]:
import os
import re

import numpy as np
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact

from src.features.univar import UnivariateAnalysis
from src.features.bivar import BivariateAnalysis


## Chargement des données

In [2]:
data = dict()
data_dir = os.path.abspath('../data/raw')
for file in os.listdir(data_dir):
    if file.endswith('.csv'):
        key = file.split('.')[0]
        data[key] = pd.read_csv(os.path.join(data_dir, file))

In [3]:
_2015 = '2015-building-energy-benchmarking'
_2016 = '2016-building-energy-benchmarking'

In [4]:
col_2015 = data[_2015].columns.values.tolist()
col_2016 = data[_2016].columns.values.tolist()
print(set(col_2016) - set(col_2015))

{'Address', 'GHGEmissionsIntensity', 'City', 'State', 'Comments', 'Longitude', 'Latitude', 'ZipCode', 'TotalGHGEmissions'}


In [5]:
to_drop = [
            'Zip Codes',
            'City Council Districts',
            'SPD Beats',
            '2010 Census Tracts',
            'Seattle Police Department Micro Community Policing Plan Areas'
           ]

In [6]:
data[_2015].drop(to_drop, axis=1, inplace=True)
columns = {'GHGEmissions(MetricTonsCO2e)': 'TotalGHGEmissions',
           'GHGEmissionsIntensity(kgCO2e/ft2)': 'GHGEmissionsIntensity',
           'Comment': 'Comments'}
data[_2015].rename(columns=columns, inplace=True)

In [7]:
location = data[_2015]['Location']
data[_2015]['Location'] = location.apply(eval)

In [8]:
data[_2015]['Latitude'] = location.apply(lambda x: float(x['latitude']))
data[_2015]['Longitude'] = location.apply(lambda x: float(x['longitude']))

In [9]:
address_2015 = data[_2015]['Location'].apply(lambda x: x['human_address'])
address_2015 = address_2015.map(eval)
for field in ['Address', 'State', 'City']:
    data[_2015][field] = address_2015.apply(lambda x: x[field.lower()])
data[_2015]['ZipCode'] = address_2015.apply(lambda x: x['zip'])

In [10]:
col_2015 = data[_2015].columns.values.tolist()
col_2016 = data[_2016].columns.values.tolist()
print(set(col_2016) - set(col_2015))
print(set(col_2015) - set(col_2016))

set()
{'OtherFuelUse(kBtu)', 'Location'}


In [11]:
data = pd.concat(data, sort=False)

In [12]:
data.rename({"2015-building-energy-benchmarking": 2015,
             "2016-building-energy-benchmarking": 2016}, inplace=True)

In [13]:
data.drop(['Location', 'DataYear'], axis=1, inplace=True)

In [14]:

def strip_all_string(x):
    if type(x) == str:
        return x.strip()
    else:
        return x


for col in data.columns:
    data[col] = data[col].apply(strip_all_string)

### Correction des types de données

In [15]:
categorical_fields = ['BuildingType', 'PrimaryPropertyType', 'Neighborhood',
                      'LargestPropertyUseType', 'SecondLargestPropertyUseType',
                      'ThirdLargestPropertyUseType']
for col in categorical_fields:
    data[col] = data[col].astype('category')

In [16]:
for col in data.columns:
    print(f"col : {col} dtype : {data[col].dtype}")

col : OSEBuildingID dtype : int64
col : BuildingType dtype : category
col : PrimaryPropertyType dtype : category
col : PropertyName dtype : object
col : TaxParcelIdentificationNumber dtype : object
col : CouncilDistrictCode dtype : int64
col : Neighborhood dtype : category
col : YearBuilt dtype : int64
col : NumberofBuildings dtype : float64
col : NumberofFloors dtype : float64
col : PropertyGFATotal dtype : int64
col : PropertyGFAParking dtype : int64
col : PropertyGFABuilding(s) dtype : int64
col : ListOfAllPropertyUseTypes dtype : object
col : LargestPropertyUseType dtype : category
col : LargestPropertyUseTypeGFA dtype : float64
col : SecondLargestPropertyUseType dtype : category
col : SecondLargestPropertyUseTypeGFA dtype : float64
col : ThirdLargestPropertyUseType dtype : category
col : ThirdLargestPropertyUseTypeGFA dtype : float64
col : YearsENERGYSTARCertified dtype : object
col : ENERGYSTARScore dtype : float64
col : SiteEUI(kBtu/sf) dtype : float64
col : SiteEUIWN(kBtu/sf) d

In [17]:
data.count(axis=0).to_dict()

{'OSEBuildingID': 6716,
 'BuildingType': 6716,
 'PrimaryPropertyType': 6716,
 'PropertyName': 6716,
 'TaxParcelIdentificationNumber': 6714,
 'CouncilDistrictCode': 6716,
 'Neighborhood': 6716,
 'YearBuilt': 6716,
 'NumberofBuildings': 6708,
 'NumberofFloors': 6708,
 'PropertyGFATotal': 6716,
 'PropertyGFAParking': 6716,
 'PropertyGFABuilding(s)': 6716,
 'ListOfAllPropertyUseTypes': 6580,
 'LargestPropertyUseType': 6560,
 'LargestPropertyUseTypeGFA': 6560,
 'SecondLargestPropertyUseType': 3238,
 'SecondLargestPropertyUseTypeGFA': 3238,
 'ThirdLargestPropertyUseType': 1156,
 'ThirdLargestPropertyUseTypeGFA': 1156,
 'YearsENERGYSTARCertified': 229,
 'ENERGYSTARScore': 5093,
 'SiteEUI(kBtu/sf)': 6699,
 'SiteEUIWN(kBtu/sf)': 6700,
 'SourceEUI(kBtu/sf)': 6697,
 'SourceEUIWN(kBtu/sf)': 6697,
 'SiteEnergyUse(kBtu)': 6701,
 'SiteEnergyUseWN(kBtu)': 6700,
 'SteamUse(kBtu)': 6697,
 'Electricity(kWh)': 6697,
 'Electricity(kBtu)': 6697,
 'NaturalGas(therms)': 6697,
 'NaturalGas(kBtu)': 6697,
 'Othe

### Localisation des bâtiments

On ne regarde que pour l'année 2016 (En principe les bâtiments contenus dans le set 2015 sont repris dans celui de 2016)

In [18]:
data.loc[2015][['Latitude', 'Longitude']].values

array([[  47.61219025, -122.33799744],
       [  47.61310583, -122.33335756],
       [  47.61334897, -122.33769944],
       ...,
       [  47.61649845, -122.33770787],
       [  47.68396954, -122.2874088 ],
       [  47.68396954, -122.2874088 ]])

In [19]:
year_widget = widgets.Dropdown(options=[2015, 2016])
usage_type = data['LargestPropertyUseType'].sort_values()
usage_type = usage_type.drop_duplicates().tolist()
usage_type.insert(0, 'ALL')
usage_type.remove(np.nan)
usage_widget = widgets.Dropdown(option=usage_type)


@interact
def make_map(year=year_widget, usage=usage_type):
    location = data.loc[year][['Latitude', 'Longitude']].mean(axis=0).values
    data_map = data.loc[year][['Latitude',
                               'Longitude',
                               'LargestPropertyUseType']]
    if usage != 'ALL':
        data_map = data_map[data_map['LargestPropertyUseType'] == usage]
    m = folium.Map(location=location,
                   tiles='cartodbpositron',
                   zoom_start=11)

    mc = FastMarkerCluster(data_map)
    mc.add_to(m)

    display(m)

interactive(children=(Dropdown(description='year', options=(2015, 2016), value=2015), Dropdown(description='us…

## Analyses univariées

In [20]:
data.columns = data.columns.map(lambda x: x.replace('(', '_'))
data.columns = data.columns.map(lambda x: x.replace(')', ''))
data.columns = data.columns.map(lambda x: x.replace('/', '_'))

dtypes = data.columns.map(lambda x: data[x].dtype.name)
opt = [f'{x} -- ({y})' for x, y in zip(data.columns.values.tolist(), dtypes)]
variable_widget = widgets.Dropdown(options=opt)


@interact
def univariate_analysis(var=variable_widget):
    univar = UnivariateAnalysis(data)
    field, dtype = map(str.strip, var.split('--'))
    dtype = re.sub(r'[\(\)]', '', dtype)
    if dtype not in ['float64', 'int64']:
        raise NotImplementedError
    univar.make_analysis(field)

interactive(children=(Dropdown(description='var', options=('OSEBuildingID -- (int64)', 'BuildingType -- (categ…

## Analyses bivariées

In [23]:

dtypes = list(map(lambda x: data[x].dtype.name, data.columns))
names_dtypes = zip(data.columns.values.tolist(), dtypes)
names_dtypes = [(x, y) for x, y in names_dtypes]

opt_1 = [x for x, y in names_dtypes if y in ['float64', ]]
opt_2 = [x for x, y in names_dtypes if y == 'category']
outcome_variable = widgets.Dropdown(options=opt_1)
group = widgets.Dropdown(options=opt_2)


@interact
def anova(outcome_variable=outcome_variable, group=group):
    bivar = BivariateAnalysis(data)
    bivar.anova(outcome_variable=outcome_variable,
                group=group,
                orient='h',
                figsize=(12, 12),
                label_rotation=0)

interactive(children=(Dropdown(description='outcome_variable', options=('NumberofBuildings', 'NumberofFloors',…

## Check point

In [22]:
data.to_pickle('../data/interim/full_data.pickle')